In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import joblib
import os
from sklearn.preprocessing import MinMaxScaler

In [11]:

# List of stock tickers to train models on
tickers = ["AAPL", "GOOGL", "MSFT", "TSLA", "AMZN", "NFLX", "META"]

# Folder to store models and scalers
if not os.path.exists("models"):
    os.makedirs("models")

for ticker in tickers:
    model_path = f"models/{ticker}_lstm.h5"
    scaler_path = f"models/{ticker}_scaler.pkl"

    if os.path.exists(model_path) and os.path.exists(scaler_path):
        print(f"Model for {ticker} already exists. Skipping training...\n")
        continue  # Skip training if model already exists

    print(f"Training model for {ticker}...")

    # -------------------- Step 1: Download Data --------------------
    start_date = "2020-01-01"
    end_date = "2024-12-31"
    df = yf.download(ticker, start=start_date, end=end_date)

    # -------------------- Step 2: Preprocess Data --------------------
    scaler = MinMaxScaler(feature_range=(0, 1))
    df["Close_Scaled"] = scaler.fit_transform(df["Close"].values.reshape(-1, 1))

    sequence_length = 60  # Use past 60 days to predict next day
    X, y = [], []
    for i in range(len(df) - sequence_length):
        X.append(df["Close_Scaled"].iloc[i : i + sequence_length].values)
        y.append(df["Close_Scaled"].iloc[i + sequence_length])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))  # Reshape for LSTM

    # -------------------- Step 3: Build LSTM Model --------------------
    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)),
        Dropout(0.2),
        LSTM(units=50, return_sequences=False),
        Dropout(0.2),
        Dense(units=25),
        Dense(units=1)  # Predicting 1 value (next day's price)
    ])

    model.compile(optimizer="adam", loss="mean_squared_error")

    # -------------------- Step 4: Train Model --------------------
    model.fit(X, y, batch_size=32, epochs=20, verbose=1)

    # -------------------- Step 5: Save Model & Scaler --------------------
    model.save(model_path)  # Save model
    joblib.dump(scaler, scaler_path)  # Save scaler
    print(f"Model for {ticker} saved!\n")

Model for AAPL already exists. Skipping training...

Model for GOOGL already exists. Skipping training...

Model for MSFT already exists. Skipping training...

Model for TSLA already exists. Skipping training...

Model for AMZN already exists. Skipping training...

Model for NFLX already exists. Skipping training...

Model for META already exists. Skipping training...



In [18]:

from tensorflow.keras.models import load_model

tickers = ["AAPL", "GOOGL", "MSFT", "TSLA", "AMZN", "NFLX", "META"]

for ticker in tickers:
    model_path = f"models/{ticker}_lstm.h5"
    scaler_path = f"models/{ticker}_scaler.pkl"

    if not os.path.exists(model_path) or not os.path.exists(scaler_path):
        print(f"⚠️ Model or scaler missing for {ticker}. Skipping...\n")
        continue

    print(f"📊 Testing model for {ticker}...\n")

    # Load model
    model = load_model(model_path)
    model.compile(optimizer="adam", loss="mean_squared_error")  # Ensure it's compiled

    # Load scaler
    scaler = joblib.load(scaler_path)

    # Get latest 70 days of data (buffer for missing days)
    df = yf.download(ticker, period="70d")

    if df.shape[0] < 60:
        print(f"⚠️ Not enough data to test {ticker}. Skipping...\n")
        continue

    # Extract the last 60 days
    last_60_days = df["Close"].values[-60:].reshape(-1, 1)
    
    # Ensure non-NaN values before scaling
    if np.isnan(last_60_days).any():
        print(f"⚠️ Data contains NaN values for {ticker}. Skipping...\n")
        continue

    # Get actual closing price of the 60th day
    last_day_price = last_60_days[-1][0]  
    print(f"📌 Actual Closing Price on 60th Day: ${last_day_price:.2f}")

    # Scale data
    last_60_days_scaled = scaler.transform(last_60_days)

    # Prepare input for LSTM
    X_test = np.array([last_60_days_scaled])
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    # Predict next day's price
    predicted_price_scaled = model.predict(X_test)
    predicted_price = scaler.inverse_transform(predicted_price_scaled)[0][0]  # Convert back to actual price

    print(f"🔮 Predicted Price for 61st Day: ${predicted_price:.2f}\n")


📊 Testing model for AAPL...



[*********************100%***********************]  1 of 1 completed


📌 Actual Closing Price on 60th Day: $240.36
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step


🔮 Predicted Price for 61st Day: $239.13

📊 Testing model for GOOGL...



[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


📌 Actual Closing Price on 60th Day: $984.69
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step


🔮 Predicted Price for 61st Day: $1026.25

📊 Testing model for META...



[*********************100%***********************]  1 of 1 completed


📌 Actual Closing Price on 60th Day: $670.78
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
🔮 Predicted Price for 61st Day: $679.98

